In [47]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_files
import os
import glob
import numpy as np
import shutil
import math
import cv2

In [66]:
root_folder = "/docs/src/kt/data_by_series_resized"
train_folder = "/docs/src/kt/data_train"
test_folder = "/docs/src/kt/data_test"
IMG_PX_SIZE = 150
IMG_HEIGHT = 150

BATCH_SIZE = 5

TRAIN_SET_SIZE = 95
EPOCHS = 5


In [67]:
def GetAllFoldersList(main_folder):
    result = []
    for patient_folder in glob.glob(main_folder + "/*"):
        for series_folder in glob.glob(patient_folder + "/*"):
            result.append(series_folder)
    
    return result

In [68]:
def scale_to_0_1(arr):
    rightMin = 0
    rightMax = 1
    leftMin = arr.min()
    leftMax = arr.max()
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = (arr - leftMin) / (leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

def ScaleImageDownTwice(arr):
    arr = np.delete(arr, list(range(0, arr.shape[0], 2)), axis=0)
    arr = np.delete(arr, list(range(0, arr.shape[1], 2)), axis=1)
    return arr


def GetPixelDataFromDCIM(image_file):
    _dataset = pydicom.dcmread(image_file)
    pixel_array = _dataset.pixel_array
    del _dataset
    return pixel_array

def GetScaledPixelDataFromDCIM(image_file):
    return scale_to_0_1(GetPixelDataFromDCIM(image_file))


# pix_data = ScaleImageDownTwice(GetScaledPixelDataFromDCIM(root_folder + "/EFREMOV___SERGEY__ALEXEEVICH_VIPROMID370-85ML/5/1.dcm"))
# print(pix_data)
# GetScaledPixelDataFromDCIM(root_folder + "/EFREMOV___SERGEY__ALEXEEVICH_VIPROMID370-85ML/5/2.dcm  "  )
# GetScaledPixelDataFromDCIM(root_folder + "/EFREMOV___SERGEY__ALEXEEVICH_VIPROMID370-85ML/5/3.dcm  "  )
# GetScaledPixelDataFromDCIM(root_folder + "/EFREMOV___SERGEY__ALEXEEVICH_VIPROMID370-85ML/5/4.dcm  "  )
# print(pix_data)


In [69]:
def Read3DImageFromFolder(folders):
    return np.random.randint(10, size=(IMG_HEIGHT, IMG_PX_SIZE, IMG_PX_SIZE, 1))
    result = []
    for folder in folders:
        file_list = []
        folder_str = folder.numpy().decode()
#         print("data:", folder_str)
        for image in glob.glob(folder_str + "/*.dcm"):
            image_number = int(image[len(folder_str) + 1:-4])
            file_list.append(image_number)
        file_list = np.sort(np.array(file_list))
        single_3d_photo = []
        for file_number in file_list:
            file_name = folder_str + "/" + str(file_number) + ".dcm"
            pixel_data = (GetScaledPixelDataFromDCIM(file_name))
            pixel_data = cv2.resize(np.array(pixel_data),(IMG_PX_SIZE,IMG_PX_SIZE))
            single_3d_photo.append(pixel_data)
        
        result = single_3d_photo
    result = np.expand_dims(result, axis=3)
    return result[:IMG_HEIGHT, ...]

# r1 = Read3DImageFromFolder(tf.constant(["/docs/src/kt/data_by_series_resized\\Znamenskaya N.G. - Body 1.0\\3", "/docs/src/kt/data_by_series_resized\\Znamenskaya N.G. - Body 1.0\\12"]))
# r1 = Read3DImageFromFolder(tf.constant(["/docs/src/kt/data_by_series_resized\\Znamenskaya N.G. - Body 1.0\\3"]))
# t1 = tf.constant(r1)
# t1.shape

In [70]:
def ReadCategoryFromFolder(folders):
    result = []
    for folder in folders:
        file_list = []
        folder_str = folder.numpy().decode()
#         print("result:", folder_str, end="")
        single_category = []
        file_name = folder_str + "/result"
        try:
            f = open(file_name, "r")
            result = int(f.read())
            f.close()
#             print(" -", result, end="")
        except IOError:
            print()
            print ("ERROR: Could not read result_file:", file_name)
#         print()
    return result

# r1 = ReadCategoryFromFolder(tf.constant(["/docs/src/kt/data_by_series_resized\\Buslaev S.N. - Body 1.0\\12"]))
# r1

In [71]:

def gen(main_folder):
    folders_list = GetAllFoldersList(main_folder.decode())
    
    folders_dataset = tf.data.Dataset.from_tensor_slices(folders_list).shuffle(buffer_size=100).repeat(EPOCHS).batch(1) # --- batch(1) uses to iterate over Dataset, DO NOT mkae it larger than 1

    for folders in folders_dataset:
        images = Read3DImageFromFolder(folders)
        category = ReadCategoryFromFolder(folders)
        yield(images, 1)
        # yield(images, folders)

ds_3d_images = tf.data.Dataset.from_generator(gen, (tf.float32, tf.float32), (tf.TensorShape([IMG_HEIGHT, IMG_PX_SIZE, IMG_PX_SIZE, 1]), tf.TensorShape(())), args=([train_folder])).batch(BATCH_SIZE)

# for i, (images, categories) in enumerate(ds_3d_images.take(300)):
#     print("epoch 1) " + str(i) + ")", images.shape, categories.shape)
# for i, (images, categories) in enumerate(ds_3d_images.take(300)):
#     print("epoch 2) " + str(i) + ")", images.shape, categories.shape)


In [72]:
input_layer = tf.keras.layers.Input((IMG_HEIGHT, IMG_PX_SIZE, IMG_PX_SIZE, 1))
# input_layer = tf.keras.layers.MaxPool3D(pool_size=(1,2,2))(input_layer)

conv_layer_1_1 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(input_layer)
pooling_layer_1 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_1_1)
pooling_layer_1 = tf.keras.layers.BatchNormalization()(pooling_layer_1)

conv_layer_2_1 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_1)
conv_layer_2_2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_2_1)
pooling_layer_2 = tf.keras.layers.MaxPool3D(pool_size=(4,4,4))(conv_layer_2_2)
pooling_layer_2 = tf.keras.layers.BatchNormalization()(pooling_layer_2)

conv_layer_3_1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_2)
conv_layer_3_2 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_3_1)
pooling_layer_3 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_3_2)
pooling_layer_3 = tf.keras.layers.BatchNormalization()(pooling_layer_3)

conv_layer_4_1 = tf.keras.layers.Conv3D(filters=128, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_3)
conv_layer_4_2 = tf.keras.layers.Conv3D(filters=128, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_4_1)
pooling_layer_4 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_4_2)
pooling_layer_4 = tf.keras.layers.BatchNormalization()(pooling_layer_4)

conv_layer_5_1 = tf.keras.layers.Conv3D(filters=256, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_4)
conv_layer_5_2 = tf.keras.layers.Conv3D(filters=256, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_5_1)
pooling_layer_5 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_5_2)
pooling_layer_5 = tf.keras.layers.BatchNormalization()(pooling_layer_5)

# conv_layer_6_1 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_5)
# conv_layer_6_2 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_6_1)
# pooling_layer_6 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_6_2)
# pooling_layer_6 = tf.keras.layers.BatchNormalization()(pooling_layer_6)

# conv_layer_7_1 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_6)
# conv_layer_7_2 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_7_1)
# pooling_layer_7 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv_layer_7_2)
# pooling_layer_7 = tf.keras.layers.BatchNormalization()(pooling_layer_7)

# conv_layer_8_1 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(pooling_layer_7)
# conv_layer_8_2 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='SAME', activation='relu')(conv_layer_8_1)
# pooling_layer_8 = tf.keras.layers.MaxPool3D(pool_size=(1,2,2))(conv_layer_8_2)
# pooling_layer_8 = tf.keras.layers.BatchNormalization()(pooling_layer_8)

flatten_layer = tf.keras.layers.Flatten()(pooling_layer_5)
dense_layer_1 = tf.keras.layers.Dense(units=512, activation='relu')(flatten_layer)
dense_layer_1 = tf.keras.layers.Dropout(0.4)(dense_layer_1)

dense_layer_2 = tf.keras.layers.Dense(units=128, activation='relu')(dense_layer_1)
dense_layer_2 = tf.keras.layers.Dropout(0.4)(dense_layer_2)

output_layer = tf.keras.layers.Dense(units=2, activation="softmax")(dense_layer_2)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 150, 150, 150, 1) 0         
_________________________________________________________________
conv3d_61 (Conv3D)           (None, 150, 150, 150, 16) 448       
_________________________________________________________________
max_pooling3d_37 (MaxPooling (None, 75, 75, 75, 16)    0         
_________________________________________________________________
batch_normalization_37 (Batc (None, 75, 75, 75, 16)    64        
_________________________________________________________________
conv3d_62 (Conv3D)           (None, 75, 75, 75, 32)    13856     
_________________________________________________________________
conv3d_63 (Conv3D)           (None, 75, 75, 75, 32)    27680     
_________________________________________________________________
max_pooling3d_38 (MaxPooling (None, 18, 18, 18, 32)    0  

In [73]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy, 
    optimizer=tf.keras.optimizers.Adadelta(lr=0.1),
#     optimizer=tf.keras.optimizers.SGD(lr=0.1),
    metrics=['acc'],
)
model.fit(
    ds_3d_images, 
#     batch_size = BATCH_SIZE,
    steps_per_epoch = int(0.75 * TRAIN_SET_SIZE / BATCH_SIZE),
    epochs=EPOCHS,
#     validation_steps= int(0.25 * TRAIN_SET_SIZE / BATCH_SIZE),
#     validation_split = 0.2,
)

Train for 14 steps
Epoch 1/5
14/14 [==============================] - 929s 66s/step - loss: 3.0442 - acc: 0.4571
Epoch 2/5
14/14 [==============================] - 938s 67s/step - loss: 6.9036 - acc: 0.5286
Epoch 3/5
14/14 [==============================] - 937s 67s/step - loss: 17.2546 - acc: 0.5571
Epoch 4/5
14/14 [==============================] - 927s 66s/step - loss: 46.2137 - acc: 0.4857
Epoch 5/5
14/14 [==============================] - 962s 69s/step - loss: 132.8734 - acc: 0.4571
